In [109]:
import random
from string import ascii_uppercase, ascii_letters
from pprint import pprint

In [2]:
## from http://users.telenet.be/d.rijmenants/en/enigmatech.htm
entry = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'  #(rotor right side)   
I     = 'EKMFLGDQVZNTOWYHXUSPAIBRCJ'
II    = 'AJDKSIRUXBLHWTMCQGZNPYFVOE'
III   = 'BDFHJLCPRTXVZNYEIWGAKMUSQO'
IV    = 'ESOVPZJAYQUIRHXLNFTGKDCMWB'
V     = 'VZBRGITYUPSDNHLXAWMJQOFECK'
## Additional rotors used by Kriegsmarine M3 and M4 only:
VI    = 'JPGVOUMFYQBENHZRDKASXLICTW'
VII   = 'NZJHGRCXMYSWBOUFAIVLPEKQDT'
VIII  = 'FKQHTLXOCBJSPDZRAMEWNIUYGV'

## reflectors, standard army & airforce
contacts   = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ' 
breflector = 'YRUHQSLDPXNGOKMIEBFZCWVJAT'
creflector = 'FVPJIAOYEDRZXWGCTKUQSBNMHL'

Notches
Rotor    Notch 	Window 	next left rotor steps when rotor steps from/to

I        Y      Q       Q -> R
II       M      E       E -> F
III      D      V       V -> W
IV       R      J       J -> K
V        H      Z       Z -> A
VI-VIII  H+U    Z, M    Z -> A and M -> N

Rotor 	Notch 	Window 	next left rotor steps when rotor steps from/to

I
II
III
IV
V
VI VII VIII 

	

Y
M
D
R
H
H + U

	

Q
E
V
J
Z
Z and M

	

Q -> R
E -> F
V -> W
J -> K
Z -> A
Z -> A and M -> N

In [3]:
raw_rotors = {'I':I, 'II':II, 'III':III, 'IV':IV, 'V':V}
print(raw_rotors)

{'I': 'EKMFLGDQVZNTOWYHXUSPAIBRCJ', 'II': 'AJDKSIRUXBLHWTMCQGZNPYFVOE', 'III': 'BDFHJLCPRTXVZNYEIWGAKMUSQO', 'IV': 'ESOVPZJAYQUIRHXLNFTGKDCMWB', 'V': 'VZBRGITYUPSDNHLXAWMJQOFECK'}


In [4]:
base_rotors = {'I':[], 'II':[], 'III':[], 'IV':[], 'V':[]}
for k in base_rotors.keys():
    base_rotors[k] = [ascii_uppercase.index(c) for c in raw_rotors[k]]
# rI = [ascii_uppercase.index(c) for c in I]
for k,v in base_rotors.items():
    print(k,':',v)

I : [4, 10, 12, 5, 11, 6, 3, 16, 21, 25, 13, 19, 14, 22, 24, 7, 23, 20, 18, 15, 0, 8, 1, 17, 2, 9]
II : [0, 9, 3, 10, 18, 8, 17, 20, 23, 1, 11, 7, 22, 19, 12, 2, 16, 6, 25, 13, 15, 24, 5, 21, 14, 4]
III : [1, 3, 5, 7, 9, 11, 2, 15, 17, 19, 23, 21, 25, 13, 24, 4, 8, 22, 6, 0, 10, 12, 20, 18, 16, 14]
IV : [4, 18, 14, 21, 15, 25, 9, 0, 24, 16, 20, 8, 17, 7, 23, 11, 13, 5, 19, 6, 10, 3, 2, 12, 22, 1]
V : [21, 25, 1, 17, 6, 8, 19, 24, 20, 15, 18, 3, 13, 7, 11, 23, 0, 22, 12, 9, 16, 14, 5, 4, 2, 10]


In [60]:
rev_rotors = {}

for r in raw_rotors.keys():
    working = {k:entry.index(v) for k,v in zip(raw_rotors[r],entry)}
    rev_rotors[r] = [working[k] for k in sorted(working.keys())]

for k,v in rev_rotors.items():
    print(k,':',v)

I : [20, 22, 24, 6, 0, 3, 5, 15, 21, 25, 1, 4, 2, 10, 12, 19, 7, 23, 18, 11, 17, 8, 13, 16, 14, 9]
II : [0, 9, 15, 2, 25, 22, 17, 11, 5, 1, 3, 10, 14, 19, 24, 20, 16, 6, 4, 13, 7, 23, 12, 8, 21, 18]
III : [19, 0, 6, 1, 15, 2, 18, 3, 16, 4, 20, 5, 21, 13, 25, 7, 24, 8, 23, 9, 22, 11, 17, 10, 14, 12]
IV : [7, 25, 22, 21, 0, 17, 19, 13, 11, 6, 20, 15, 23, 16, 2, 4, 9, 12, 1, 18, 10, 3, 24, 14, 8, 5]
V : [16, 2, 24, 11, 23, 22, 4, 13, 5, 19, 25, 14, 18, 12, 21, 9, 20, 3, 10, 6, 8, 0, 17, 15, 7, 1]


In [6]:
reflectors = {'Brf': {k:v for k,v in zip(entry, breflector)}, 'Crf': {k:v for k,v in zip(entry, creflector)}}
print(reflectors)

{'Brf': {'A': 'Y', 'B': 'R', 'C': 'U', 'D': 'H', 'E': 'Q', 'F': 'S', 'G': 'L', 'H': 'D', 'I': 'P', 'J': 'X', 'K': 'N', 'L': 'G', 'M': 'O', 'N': 'K', 'O': 'M', 'P': 'I', 'Q': 'E', 'R': 'B', 'S': 'F', 'T': 'Z', 'U': 'C', 'V': 'W', 'W': 'V', 'X': 'J', 'Y': 'A', 'Z': 'T'}, 'Crf': {'A': 'F', 'B': 'V', 'C': 'P', 'D': 'J', 'E': 'I', 'F': 'A', 'G': 'O', 'H': 'Y', 'I': 'E', 'J': 'D', 'K': 'R', 'L': 'Z', 'M': 'X', 'N': 'W', 'O': 'G', 'P': 'C', 'Q': 'T', 'R': 'K', 'S': 'U', 'T': 'Q', 'U': 'S', 'V': 'B', 'W': 'N', 'X': 'M', 'Y': 'H', 'Z': 'L'}}


In [116]:
def once_thru_scramble(ch, direction='forward', first_rotor='I', pos1=0, second_rotor='II', pos2=0, 
                       third_rotor='III', pos3=0):
    """ ch must be single ASCII character A-Z
    direction is either 'forward' or 'back' """
    if direction == 'forward':
        usedict = {k:v for k,v in base_rotors.items()}
    elif direction == 'back':
        usedict = {k:v for k,v in rev_rotors.items()}
    else:
        print('only forward or back for direction')
        return 'wtf'
    
    entry = ascii_uppercase
    ch = ch.upper()
    entry_pos = entry.index(ch)
    fst_pos_mod = (26 + pos1 - 0)%26
#     print(fst_pos_mod)
    fst_in = (entry_pos + fst_pos_mod)%26
#     print(fst_in)
    fst_out = usedict[first_rotor][fst_in]
#     print(fst_out)
    ch1o = entry[fst_out]
#     print('first rotor:  ', ch, '-->', ch1o)
    
    scd_pos_mod = (26 + pos2 - pos1)%26
    scd_in = (fst_out + scd_pos_mod)%26
    ch2i = entry[scd_in]
#     print('r1 to r2:     ', ch1o, '-->', ch2i)
#     print(scd_pos_mod, scd_in)
    scd_out = usedict[second_rotor][scd_in]
#     print(scd_out)
    ch2o = entry[scd_out]
#     print('second rotor: ', ch2i, '-->', ch2o)
    
    thd_pos_mod = (26 + pos3 - pos2)%26
    thd_in = (scd_out + thd_pos_mod)%26
    ch3i = entry[thd_in]
#     print('r2 to r3:     ', ch2o, '-->', ch3i)
    thd_out = usedict[third_rotor][thd_in]
#     print(thd_out)
    ch3o = entry[thd_out]
#     print('third rotor:  ', ch3i, '-->', ch3o)
    
    
    if direction == 'forward':
        print(f"{ch} -> (RR out) {ch1o} -> (MR in) {ch2i} -> (MR out) {ch2o} -> (LR in) {ch3i} -> (LR out) {ch3o}")
    elif direction == 'back':
        print(f"{ch} -> (LR out) {ch1o} -> (MR in) {ch2i} -> (MR out) {ch2o} -> (RR in) {ch3i} -> (RR out) {ch3o}")
    
    
    return ch3o
    
    
once_thru_scramble('q')

Q -> (RR out) X -> (MR in) X -> (MR out) V -> (LR in) V -> (LR out) M


'M'

In [122]:
def full_scramble(in_ch, left_rotor='III', lpos=0, middle_rotor='II', mpos=0, right_rotor='I', rpos=0, rflector='Brf'):
    entry = ascii_uppercase
    in_ch = in_ch.upper()
    
    ## first run right to left through scrambler
    forward_run = once_thru_scramble(in_ch, 'forward', first_rotor=right_rotor, pos1=rpos, second_rotor=middle_rotor,
                                     pos2=mpos, third_rotor=left_rotor, pos3=lpos)
    
    ## reflector back around for return
    rfi_pos_mod = (26 + 0 - lpos)%26    ## the '0' is there to matching formatting of other position modifiers - reflector is not moved so it will always be 0
    rf_in = (entry.index(forward_run) + rfi_pos_mod)%26
    chri = entry[rf_in]
    mirrored = reflectors[rflector][chri]
    
    print(f"{forward_run} -> {chri} (into reflector) -> {mirrored} (reflected out)")
    
    ## second run back left to right thru scrambler
    back_run = once_thru_scramble(mirrored, direction='back', first_rotor=left_rotor, pos1=lpos, 
                                  second_rotor=middle_rotor, pos2=mpos, third_rotor=right_rotor, pos3=rpos)
    
    bk_out = entry.index(back_run)
    bko_pos_mod = (26 + 0 - rpos)%26  ## as above, '0' just reflects that the entry interface doesn't move
    bk_final = (bk_out + bko_pos_mod)%26
    final = entry[bk_final]
    print('RR back out:  ', back_run, '-->', final)
    
    return final

# full_scramble('i')

In [123]:
full_scramble('q', lpos=2, mpos=22, rpos=10)

Q -> (RR out) E -> (MR in) Q -> (MR out) Q -> (LR in) W -> (LR out) U
U -> S (into reflector) -> F (reflected out)
F -> (LR out) D -> (MR in) X -> (MR out) I -> (RR in) W -> (RR out) N
RR back out:   N --> D


'D'

In [125]:
word = 'oh what a beautiful morning oh what a beautiful day i have a wonderful feeling everthings going my way'

In [124]:
def step_rotor(p_in):
    """"""
    if p_in == 25:
        p_in = 0
    else:
        p_in += 1
    return p_in

In [133]:
def only_ascii(instring):
    """strips out anything that's not an ascii character (i.e a-z alphabet character), capitalises"""
    newstring = ''
    for character in instring:
        if character in ascii_letters:
            newstring += character.upper()
    return newstring

In [138]:
only_ascii(word)

'OHWHATABEAUTIFULMORNINGOHWHATABEAUTIFULDAYIHAVEAWONDERFULFEELINGEVERTHINGSGOINGMYWAY'

In [147]:
sht = 'pat'
sht = sht.upper()

lt, mt, rt = (ascii_uppercase.index(s) for s in sht)
print(lt, mt, rt)

15 0 19


In [150]:
rt

19

In [152]:
def enigmatise(tocode, startset='AAA', lstartpos=0, mstartpos=0, rstartpos=0):
    encoded = ''

    tocode = only_ascii(tocode)
    
    lftpos, midpos, rgtpos = (ascii_uppercase.index(s) for s in startset.upper())
    
    
    # start out initial positions as starting positions
#     lftpos = lstartpos
#     midpos = mstartpos
#     rgtpos = rstartpos

    ## TO DO need to enable notches to be defined by ring setting as input to function
    mnotch = 4   ## point if right rotor reaches will trigger middle rotor to step
    lnotch = 5  ## point if middle rotor reaches will trigger left rotor to step

    for c in tocode:
        print('in:    ', c)

        ## simple stepping version - notches step M & L straight away
#         rgtpos = step_rotor(rgtpos)
#         if rgtpos == mnotch:
#             midpos = step_rotor(midpos)
#             print('--- middle rotor step ---')
#             if midpos == lnotch:
#                 lftpos = step_rotor(lftpos)
#                 print('---  left rotor step  ---')
                
        rgtpos = step_rotor(rgtpos)
        
        if rgtpos == mnotch:
            
            midpos = step_rotor(midpos)
            print('--- middle rotor step ---')
            if midpos == lnotch:
                lftpos = step_rotor(lftpos)
                print('---  left rotor step  ---')

        print('rp = ', rgtpos, 'mp = ', midpos, 'lp = ', lftpos)
        out = full_scramble(c, lpos=lftpos, mpos=midpos, rpos=rgtpos)
        print('out:   ', out,'\n')
        encoded += out

    return encoded

In [159]:
igged = enigmatise(word, startset='pat')
igged

in:     O
rp =  20 mp =  0 lp =  15
O -> (RR out) V -> (MR in) B -> (MR out) J -> (LR in) Y -> (LR out) Q
Q -> B (into reflector) -> R (reflected out)
R -> (LR out) S -> (MR in) D -> (MR out) C -> (RR in) W -> (RR out) N
RR back out:   N --> T
out:    T 

in:     H
rp =  21 mp =  0 lp =  15
H -> (RR out) M -> (MR in) R -> (MR out) G -> (LR in) V -> (LR out) M
M -> X (into reflector) -> J (reflected out)
J -> (LR out) O -> (MR in) Z -> (MR out) S -> (RR in) N -> (RR out) K
RR back out:   K --> P
out:    P 

in:     W
rp =  22 mp =  0 lp =  15
W -> (RR out) S -> (MR in) W -> (MR out) F -> (LR in) U -> (LR out) K
K -> V (into reflector) -> W (reflected out)
W -> (LR out) F -> (MR in) Q -> (MR out) Q -> (RR in) M -> (RR out) C
RR back out:   C --> G
out:    G 

in:     H
rp =  23 mp =  0 lp =  15
H -> (RR out) L -> (MR in) O -> (MR out) M -> (LR in) B -> (LR out) D
D -> O (into reflector) -> M (reflected out)
M -> (LR out) A -> (MR in) L -> (MR out) K -> (RR in) H -> (RR out) P
RR back out

'TPGSNGIIYXBRFROZDZVWRPYZIFPPEXHNLMVSRPXKETQBWCLUSKFRSGMRKKHIZJQCDKQLMPEIOZKSPPTHPQKP'

In [160]:
igged

'TPGSNGIIYXBRFROZDZVWRPYZIFPPEXHNLMVSRPXKETQBWCLUSKFRSGMRKKHIZJQCDKQLMPEIOZKSPPTHPQKP'

In [162]:
reword = enigmatise(igged, startset='pat')
reword

in:     T
rp =  20 mp =  0 lp =  15
T -> (RR out) W -> (MR in) C -> (MR out) D -> (LR in) S -> (LR out) G
G -> R (into reflector) -> B (reflected out)
B -> (LR out) Y -> (MR in) J -> (MR out) B -> (RR in) V -> (RR out) I
RR back out:   I --> O
out:    O 

in:     P
rp =  21 mp =  0 lp =  15
P -> (RR out) N -> (MR in) S -> (MR out) Z -> (LR in) O -> (LR out) Y
Y -> J (into reflector) -> X (reflected out)
X -> (LR out) V -> (MR in) G -> (MR out) R -> (RR in) M -> (RR out) C
RR back out:   C --> H
out:    H 

in:     G
rp =  22 mp =  0 lp =  15
G -> (RR out) M -> (MR in) Q -> (MR out) Q -> (LR in) F -> (LR out) L
L -> W (into reflector) -> V (reflected out)
V -> (LR out) U -> (MR in) F -> (MR out) W -> (RR in) S -> (RR out) S
RR back out:   S --> W
out:    W 

in:     S
rp =  23 mp =  0 lp =  15
S -> (RR out) H -> (MR in) K -> (MR out) L -> (LR in) A -> (LR out) B
B -> M (into reflector) -> O (reflected out)
O -> (LR out) B -> (MR in) M -> (MR out) O -> (RR in) L -> (RR out) E
RR back out

'OHWHATABEAUTIFULMORNINGOHWHATABEAUTIFULDAYIHAVEAWONDERFULFEELINGEVERTHINGSGOINGMYWAY'

In [163]:
print(igged)
print(reword)

TPGSNGIIYXBRFROZDZVWRPYZIFPPEXHNLMVSRPXKETQBWCLUSKFRSGMRKKHIZJQCDKQLMPEIOZKSPPTHPQKP
OHWHATABEAUTIFULMORNINGOHWHATABEAUTIFULDAYIHAVEAWONDERFULFEELINGEVERTHINGSGOINGMYWAY


In [248]:
crib = 'OHWHATABEAUTIFULDAY'
lncrib = len(crib)

In [249]:
reword[23:23+lncrib]

'OHWHATABEAUTIFULDAY'

In [250]:
igged[:len(igged)-lncrib]

'TPGSNGIIYXBRFROZDZVWRPYZIFPPEXHNLMVSRPXKETQBWCLUSKFRSGMRKKHIZJQCD'

In [251]:
status_of_crib_indexes = {k:True for k in range(len(igged))}
# print(status_of_crib_indexes)
for num, letter in enumerate(igged[:len(igged)-lncrib]):
    trange = range(num,num+lncrib)
    tcrib = igged[num:num+lncrib]
    for c, t in zip(crib, tcrib):
        if c == t:
            status_of_crib_indexes[num] = False
    
        
print(status_of_crib_indexes)
maybes = [k for k,v in status_of_crib_indexes.items() if v == True]
print(maybes)

{0: False, 1: True, 2: True, 3: False, 4: False, 5: True, 6: True, 7: True, 8: True, 9: True, 10: True, 11: True, 12: False, 13: True, 14: False, 15: True, 16: True, 17: False, 18: True, 19: True, 20: False, 21: True, 22: True, 23: True, 24: True, 25: True, 26: True, 27: False, 28: True, 29: False, 30: False, 31: False, 32: False, 33: False, 34: True, 35: True, 36: False, 37: False, 38: True, 39: True, 40: True, 41: True, 42: False, 43: True, 44: True, 45: True, 46: True, 47: False, 48: False, 49: True, 50: True, 51: True, 52: False, 53: True, 54: True, 55: False, 56: True, 57: False, 58: True, 59: False, 60: True, 61: True, 62: False, 63: True, 64: True, 65: True, 66: True, 67: True, 68: True, 69: True, 70: True, 71: True, 72: True, 73: True, 74: True, 75: True, 76: True, 77: True, 78: True, 79: True, 80: True, 81: True, 82: True, 83: True}
[1, 2, 5, 6, 7, 8, 9, 10, 11, 13, 15, 16, 18, 19, 21, 22, 23, 24, 25, 26, 28, 34, 35, 38, 39, 40, 41, 43, 44, 45, 46, 49, 50, 51, 53, 54, 56, 58, 

In [253]:
mcrib = igged[23:23+lncrib]
print(crib)
print(mcrib)

OHWHATABEAUTIFULDAY
ZIFPPEXHNLMVSRPXKET


In [268]:
pairs = {i:{c,m} for i,c,m in zip(range(23+lncrib), crib, mcrib)}
pairs

{0: {'O', 'Z'},
 1: {'H', 'I'},
 2: {'F', 'W'},
 3: {'H', 'P'},
 4: {'A', 'P'},
 5: {'E', 'T'},
 6: {'A', 'X'},
 7: {'B', 'H'},
 8: {'E', 'N'},
 9: {'A', 'L'},
 10: {'M', 'U'},
 11: {'T', 'V'},
 12: {'I', 'S'},
 13: {'F', 'R'},
 14: {'P', 'U'},
 15: {'L', 'X'},
 16: {'D', 'K'},
 17: {'A', 'E'},
 18: {'T', 'Y'}}

In [257]:
links = {character:len([pair for pair in pairs.values() if character in pair]) for character in entry}   
print(links)
hilinks = {k:v for k,v in links.items() if v > 1}
hilinks

{'A': 4, 'B': 1, 'C': 0, 'D': 1, 'E': 3, 'F': 2, 'G': 0, 'H': 3, 'I': 2, 'J': 0, 'K': 1, 'L': 2, 'M': 1, 'N': 1, 'O': 1, 'P': 3, 'Q': 0, 'R': 1, 'S': 1, 'T': 3, 'U': 2, 'V': 1, 'W': 1, 'X': 2, 'Y': 1, 'Z': 1}


{'A': 4,
 'E': 3,
 'F': 2,
 'H': 3,
 'I': 2,
 'L': 2,
 'P': 3,
 'T': 3,
 'U': 2,
 'X': 2}

In [273]:
pairs

{0: {'O', 'Z'},
 1: {'H', 'I'},
 2: {'F', 'W'},
 3: {'H', 'P'},
 4: {'A', 'P'},
 5: {'E', 'T'},
 6: {'A', 'X'},
 7: {'B', 'H'},
 8: {'E', 'N'},
 9: {'A', 'L'},
 10: {'M', 'U'},
 11: {'T', 'V'},
 12: {'I', 'S'},
 13: {'F', 'R'},
 14: {'P', 'U'},
 15: {'L', 'X'},
 16: {'D', 'K'},
 17: {'A', 'E'},
 18: {'T', 'Y'}}

In [290]:
hipairs = {}
for character in hilinks.keys():
    hset = {k:list(pair) for k,pair in pairs.items() if character in pair}
    newresult = {}
    for k,v in hset.items():
        v.remove(character)
        if v[0] in hilinks.keys():
            newresult[k] = v[0]
    print(hset)
    print(newresult)
    if len(newresult) > 0:
        hipairs[character] = newresult
#     hipairs[character] = {k:pair.remove(character) for k,pair in pairs.items() if character in pair}
## maybe try a comprehension again later, involves a few steps...

    
hipairs

{4: ['P'], 6: ['X'], 9: ['L'], 17: ['E']}
{4: 'P', 6: 'X', 9: 'L', 17: 'E'}
{5: ['T'], 8: ['N'], 17: ['A']}
{5: 'T', 17: 'A'}
{2: ['W'], 13: ['R']}
{}
{1: ['I'], 3: ['P'], 7: ['B']}
{1: 'I', 3: 'P'}
{1: ['H'], 12: ['S']}
{1: 'H'}
{9: ['A'], 15: ['X']}
{9: 'A', 15: 'X'}
{3: ['H'], 4: ['A'], 14: ['U']}
{3: 'H', 4: 'A', 14: 'U'}
{5: ['E'], 11: ['V'], 18: ['Y']}
{5: 'E'}
{10: ['M'], 14: ['P']}
{14: 'P'}
{6: ['A'], 15: ['L']}
{6: 'A', 15: 'L'}


{'A': {4: 'P', 6: 'X', 9: 'L', 17: 'E'},
 'E': {5: 'T', 17: 'A'},
 'H': {1: 'I', 3: 'P'},
 'I': {1: 'H'},
 'L': {9: 'A', 15: 'X'},
 'P': {3: 'H', 4: 'A', 14: 'U'},
 'T': {5: 'E'},
 'U': {14: 'P'},
 'X': {6: 'A', 15: 'L'}}

In [289]:
len(hipairs['F'])

0